This script takes subcluster_stats.csv file (emailed from Dr. Saunders) and reformat subcluster name to the same format in the annotation file, therefore mapping the number of cells to metacell names. 

In [67]:
import numpy as np
import pandas as pd

In [68]:
stats_df = pd.read_csv('../data/single-cell/saunders_2018/subcluster_stats.csv')

In [69]:
stats_df.drop(stats_df.columns[0],axis=1,inplace=True)

In [70]:
tissue_dict={
    'P60Cerebellum_ALT':'CB',
    'P60Cortex_noRep5_FRONTALonly':'FC',
    'P60Cortex_noRep5_POSTERIORonly':'PC',
    'P60EntoPeduncular':'ENT',
    'P60GlobusPallidus':'GP',
    'P60Hippocampus':'HC',
    'P60Striatum':'STR',
    'P60SubstantiaNigra':'SN',
    'P60Thalamus':'TH'
}

In [71]:
stats_df['tissue_abbrev']=stats_df['tissue'].apply(lambda x: tissue_dict.get(x))

In [72]:
stats_df['tissue_subcluster']=stats_df['tissue_abbrev']+'_'+stats_df['subcluster']

In [73]:
annotation_df=pd.read_excel('../data/single-cell/saunders_2018/annotation.BrainCellAtlas_Saunders_version_2018.04.01.xlsx')

/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [74]:
class_marker_dict={
     'Gad1Gad2': 'GABA',
     'Slc17a7': 'Glutamate',
     'Slc17a6': 'Glutamate',
     'Slc17a7-Slc17a6': 'Glutamate',
     'Th': 'Dopamine',
     'Gad1Gad2_Htr3a': 'GABA', # Htr3a+ GABAergic neurons
     'Slc17a7Slc17a6': 'Glutamate',
     'Gad1Gad2_Sst': 'GABA', # OLM neurons; SST is an OLM marker
     'Gad1Gad2-Th': 'Dopamine', # MGE-derived cortical interneurons, Striatum Th+ SPN
     'Gad1Gad2_Id2': 'GABA',
     'Gad1Gad2Slc17a8_Htr3a': 'Mix',
     'Gad1Gad2_Pvalb': 'GABA',
     'Slc17a7-Slc17a8': 'Glutamate',
     'Gad1Gad2-Chat': 'Acetylcholine',
     'Chat-Gad1Gad2-Slc17a8': 'Acetylcholine',
     'Gad1Gad2-Slc17a6': 'Mix', # e.g., Entopeduncular Nucleus (Ent), GABA/Glu co-releasing
     'Slc17a6-Slc17a7': 'Glutamate',
     'Gad1Gad2_Cplx3': 'GABA',
     'Gad1Gad2-Slc17a8': 'Mix',
     'Slc17a6-Slc17a7-Slc17a8': 'Glutamate',
     'Gad1Gad2Chat_Htr3a': 'Acetylcholine',
     'Gad1Gad2Slc17a8_Sst': 'Mix', # OLM neurons with co-releasing? 
     'Gad1Gad2-Chat-Slc17a8': 'Acetylcholine',
     'Slc17a6-Gad2-Th': 'Dopamine',
     'Th-Slc17a6': 'Dopamine',
     'Slc17a6-Gad1Gad2': 'Mix'
}

exc_inh_dict={
     'Gad1Gad2': 'inh',
     'Slc17a7': 'exc',
     'Slc17a6': 'exc',
     'Slc17a7-Slc17a6': 'exc',
     'Th': 'both',
     'Gad1Gad2_Htr3a': 'inh',
     'Slc17a7Slc17a6': 'exc',
     'Gad1Gad2_Sst': 'inh', # GABAergic OLM neurons 
     'Gad1Gad2-Th': 'both',
     'Gad1Gad2_Id2': 'inh',
     'Gad1Gad2Slc17a8_Htr3a': 'both',
     'Gad1Gad2_Pvalb': 'inh',
     'Slc17a7-Slc17a8': 'exc',
     'Gad1Gad2-Chat': 'both',
     'Chat-Gad1Gad2-Slc17a8': 'both',
     'Gad1Gad2-Slc17a6': 'both',
     'Slc17a6-Slc17a7': 'exc',
     'Gad1Gad2_Cplx3': 'inh',
     'Gad1Gad2-Slc17a8': 'both',
     'Slc17a6-Slc17a7-Slc17a8': 'exc',
     'Gad1Gad2Chat_Htr3a': 'both',
     'Gad1Gad2Slc17a8_Sst': 'both', # co-releasing OLM? 
     'Gad1Gad2-Chat-Slc17a8': 'both',
     'Slc17a6-Gad2-Th': 'both',
     'Th-Slc17a6': 'both',
     'Slc17a6-Gad1Gad2': 'both'
}

In [75]:
annotation_df['neurotransmitter']=annotation_df['class_marker'].apply(lambda x: class_marker_dict.get(x, 'Non-neuronal'))
annotation_df['exc_inh']=annotation_df['class_marker'].apply(lambda x: exc_inh_dict.get(x, 'Non-neuronal'))

In [76]:
annotation_df.drop(columns=['tissue','subcluster','full_name'],inplace=True)

In [77]:
stats_df.drop(columns=['cluster','subcluster'],inplace=True)

In [78]:
# Check whether the file Dr. Saunders sent me matches the annotation file from DropViz
# annotation_df.merge(stats_df,on='tissue_subcluster').to_excel('../results/single-cell/saunders_2018/check_match.xlsx')
# They Match! 
# Merge new dataframe
merge_df=pd.merge(left=stats_df,right=annotation_df,left_on='tissue_subcluster',right_on='tissue_subcluster',how='inner')

In [79]:
merge_df

,tissue,cluster_name,full_name,num_cells,tissue_abbrev,tissue_subcluster,class,class_marker,type_marker,common_name,neurotransmitter,exc_inh
0,P60Cerebellum_ALT,GranularNeuron_Gabra6,Neuron.Slc17a7.Gabra6,10243.0,CB,CB_1-1,NEURON,Slc17a7,Gabra6,Granule cells,Glutamate,exc
1,P60Cerebellum_ALT,PurkinjeNeuron_Pcp2,Neuron.Gad1Gad2.Pcp2,89.0,CB,CB_2-1,NEURON,Gad1Gad2,Pcp2,Purkinje Neurons,GABA,inh
2,P60Cerebellum_ALT,Interneurons_Pvalb,Neuron.Gad1Gad2.Pvalb-Nefh,360.0,CB,CB_3-1,NEURON,Gad1Gad2,Pvalb-Nefh,Cerebellum basket cells 1,GABA,inh
3,P60Cerebellum_ALT,Interneurons_Pvalb,Neuron.Gad1Gad2.Pvalb-Rab3b,196.0,CB,CB_3-2,NEURON,Gad1Gad2,Pvalb-Rab3b,Cerebellum basket cells 2,GABA,inh
4,P60Cerebellum_ALT,Interneurons_Pvalb,Neuron.Gad1Gad2.Pvalb-Cadm1,101.0,CB,CB_3-3,NEURON,Gad1Gad2,Pvalb-Cadm1,Cerebellum basket cells 3,GABA,inh
...,...,...,...,...,...,...,...,...,...,...,...,...
559,P60Thalamus,Astrocyte_Gja1,Astrocyte.Gja1.Vegfa,901.0,TH,TH_11-2,ASTROCYTE,Gja1,Vegfa,Astrocyte.Gja1.Vegfa,Non-neuronal,Non-neuronal
560,P60Thalamus,Polydendrocyte_Tnr,Polydendrocyte.Tnr.Pdgfa,113.0,TH,TH_12-1,POLYDENDROCYTE,Tnr,Pdgfa,Polydendrocyte.Tnr.Pdgfa,Non-neuronal,Non-neuronal
561,P60Thalamus,Polydendrocyte_Tnr,Polydendrocyte.Tnr.Ctps,108.0,TH,TH_12-2,POLYDENDROCYTE,Tnr,Ctps,Polydendrocyte.Tnr.Ctps,Non-neuronal,Non-neuronal
562,P60Thalamus,Polydendrocyte_Tnr,Polydendrocyte.Tnr.Bmp4,159.0,TH,TH_12-3,POLYDENDROCYTE,Tnr,Bmp4,Polydendrocyte.Tnr.Bmp4,Non-neuronal,Non-neuronal


In [80]:
merge_df.to_excel('../results/single-cell/saunders_2018/enhanced_annotation.xlsx',index=False)